# But du projet

Ce code utilise l'API Roboflow via (python 3.12) pour télécharger la dataset de 3 personnages de dragon ball :
- Goku
- Vegeta
- Freezer

Executer cette commande pour installer la bibliothèque roboflow
```
pip install roboflow
```

# Installation des bibliothèques nécessaires

In [1]:
# Tous les imports nécessaires
import shutil
from roboflow import Roboflow
import os
import random
import cv2
from ultralytics import YOLO, settings
import torch
import stat

settings.update(datasets_dir='.') # on definit le dossier de sauvegarde des datasets


# Téléchargement du dataset via l'API Roboflow si elle n'est pas déjà présente

In [2]:
# Exemple pour modifier les permissions du répertoire courant
current_dir = os.getcwd()

# Donner des permissions d'écriture au répertoire courant (777 donne des permissions complètes)
os.chmod(current_dir, stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO)

# Initialiser l'objet Roboflow avec une clé API
rf = Roboflow(api_key="aVyMuIoKSec5dyUaxD6H")

# Accéder à un projet spécifique dans un espace de travail
project = rf.workspace("converter-sq0eo").project("dbf-3hywn")

# Sélectionner une version spécifique du projet
version = project.version(5)

# Télécharger le dataset au format YOLOv8
def download_dataset():
    dataset = version.download("yolov8")

# Vérifier si un dossier contient les chaînes "dbf" et "yolo"
def check_dataset_directory():
    
    # Récupérer le répertoire courant
    current_dir = os.getcwd()
    
    # Parcourir les dossiers du répertoire courant
    for dir_name in os.listdir(current_dir):
        if "dbf" in dir_name.lower() and "5" in dir_name.lower():
            dataset_dir = os.path.join(current_dir, dir_name)
            if os.path.isdir(dataset_dir):
                required_dirs = ["test", "train", "valid"]
                required_files = ["data.yaml", "README.dataset.txt", "README.roboflow.txt"]
                for req_dir in required_dirs:
                    if not os.path.isdir(os.path.join(dataset_dir, req_dir)):
                        shutil.rmtree(dataset_dir)
                        return False
                for req_file in required_files:
                    if not os.path.isfile(os.path.join(dataset_dir, req_file)):
                        shutil.rmtree(dataset_dir)
                        return False
                return True
    return False

# Vérifier et télécharger si nécessaire
if not check_dataset_directory():
    download_dataset()
else:
    print("Le dataset est déjà présent et complet.")

loading Roboflow workspace...
loading Roboflow project...
Le dataset est déjà présent et complet.


# Entraînement du modèle YOLOv8 si le fichier yolov8n-final.pt n'existe pas

Si ce n'est pas déjà fait, l'entrainement durerait en moyenne 30 minutes avec un GPU, alors que sans GPU, cela prendrait plus longtemps.

In [3]:
# Définir le périphérique en fonction de la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique: {device}")

# Charger le modèle YOLOv8 et déplacer sur le GPU si disponible

name_file = "yolov8n-final.pt"

model = None

if not os.path.exists(name_file):
    model = YOLO('yolov8n.pt').to(device)
    # Charger les données d'entraînement
    data_yaml_path = 'DBF-5/data.yaml'

    # Vérifier si le fichier data.yaml existe
    if not os.path.exists(data_yaml_path):
        raise FileNotFoundError(f"Le fichier {data_yaml_path} est introuvable.")

    print("Chargement des données d'entraînement...")
    try:
        model.train(data=data_yaml_path, epochs=10, device=device, batch=8)
        print("Entraînement terminé.")
        model.save(name_file)
        print("Model sauvegardé avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle: {e}")
else:
    print("Model déjà existant")
    model = YOLO(name_file).to(device)

Utilisation du périphérique: cuda


100%|██████████| 6.25M/6.25M [00:00<00:00, 77.3MB/s]


Chargement des données d'entraînement...
New https://pypi.org/project/ultralytics/8.3.49 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=DBF-5/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

100%|██████████| 5.35M/5.35M [00:00<00:00, 58.9MB/s]


AMP: checks passed 


...

      10/10      1.19G     0.3515     0.3353     0.9099          1        640: 100%|██████████| 1961/1961 [02:43<00:00, 12.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:05<00:00, 11.66it/s]

                   all        983       1153      0.813      0.738      0.807      0.463



10 epochs completed in 0.485 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.28  Python-3.12.2 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:05<00:00, 11.90it/s]


                   all        983       1153      0.815      0.739      0.807      0.462
               Freezer        149        174      0.811      0.787      0.834      0.511
                  Goku        726        797      0.901      0.738      0.878      0.482
                Vegeta        171        182      0.735      0.692      0.711      0.394
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train
Entraînement terminé.
Model sauvegardé avec succès.


# Utilisation du modèle YOLOv8 pour la détection en temps réel

Ce code permet de prendre un fichier vidéo aléatoire dans le dossier mp4 et d'effectuer la détection d'objets en temps réel sur cette vidéo.

In [4]:
# Sélectionner un fichier MP4 aléatoire dans le dossier mp4
mp4_dir = 'mp4'
mp4_files = [f for f in os.listdir(mp4_dir) if f.endswith('.mp4')]
random_mp4 = random.choice(mp4_files)
video_path = os.path.join(mp4_dir, random_mp4)

print(f"Utilisation du fichier vidéo: {video_path}")

# Ouvrir la vidéo et effectuer la détection
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Effectuer la détection directement sur le frame (sans conversion en tenseur)
    results = model(frame)

    # Traiter les résultats
    for result in results:
        for box in result.boxes:
            try:
                x1, y1, x2, y2 = box.xyxy[0]
                label = result.names[int(box.cls)]
                confidence = box.conf[0]
            except (ValueError, IndexError, AttributeError) as e:
                print(f"Erreur lors de la lecture des coordonnées: {e}")
                continue
            # Dessiner le rectangle et le label sur le frame
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (int(x1), int(y1) + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Afficher le frame dans une fenêtre OpenCV
    cv2.imshow('YOLO Detection', frame)

    # Quitter si 'q' est pressé
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Utilisation du fichier vidéo: mp4\vegeta_vs_freezer.mp4

0: 384x640 (no detections), 38.7ms
Speed: 1.3ms preprocess, 38.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 3.0ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detecti